In [2]:
cd /data/store01/data_dirs/students/ucfafyi/S2_MODIS

/data/store01/data_dirs/students/ucfafyi/S2_MODIS


In [3]:
import sys
sys.path.insert(0, 'python')
from geo_trans import *

In [14]:
import numpy as np
from osgeo import ogr
from osgeo import osr
import mgrs
from scipy.interpolate import griddata
#import multiprocessing 
#from functools import partial

m_lon0, m_lon1, m_lat0, m_lat1 = -20015109.354,20015109.354,10007554.677,-10007554.677 # the defalt!!!

def transform(a=True):
    # from prof. lewis
    wgs84 = osr.SpatialReference( ) # Define a SpatialReference object
    wgs84.ImportFromEPSG( 4326 ) # And set it to WGS84 using the EPSG code
    modis_sinu = osr.SpatialReference() # define the SpatialReference object
    # In this case, we get the projection from a Proj4 string
    modis_sinu.ImportFromProj4 ( \
                    "+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")
    if a:
        # from modis to wgs 84
        tx = osr.CoordinateTransformation(modis_sinu, wgs84)
    else:
        # from wgs 84 to modis
        tx = osr.CoordinateTransformation(wgs84,modis_sinu)
    
    return tx

'''
def get_mextend():
    
    # get the begining and ending of the MODIS grid
    tx = transform(False)
    
    m_lon1, modis_lat, modis_z = tx.TransformPoint (180, 0)
    m_lon0, modis_lat, modis_z = tx.TransformPoint (-180, 0)
    modis_lon, m_lat0, modis_z = tx.TransformPoint (0, 90)
    modis_lon, m_lat1, modis_z = tx.TransformPoint (0, -90)
    
    return (m_lon0, m_lon1, m_lat0, m_lat1)
'''

def get_steps():
    # find out the modis tile step
    lon_step = (m_lon1-m_lon0)/36.
    lat_step = (m_lat1-m_lat0)/18.
    # find out the modis pixel step 
    lon_cstep = lon_step/2400.
    lat_cstep = lon_step/2400.
    
    return (lon_step, lat_step, lon_cstep, lat_cstep)




def get_wgs(h, v):
    # calculate the lat and lon for one MODIS tile
    tx = transform(True)
    
    lon_step, lat_step, lon_cstep, lat_cstep = get_steps()

    h_0 = (h)*lon_step + m_lon0 ; v_0 = m_lat0 + (v)*lat_step
    h_e = (h+1)*lon_step + m_lon0; v_e = m_lat0 + (v+1)*lat_step 

    f = lambda x: float('%.3f'%x)
    h_0, h_e, v_0, v_e = f(h_0), f(h_e), f(v_0), f(v_e)

    #hs = np.arange(h_0, h_e, lon_cstep)
    #vs = np.arange(v_e-lat_cstep, v_0, lat_cstep)[::-1]
    #h_array = np.tile(hs, 2400).reshape(2400,2400).ravel()
    #v_array = (np.tile(vs, 2400).reshape(2400,2400).T).ravel()

    # the coordinates should be related the pixel position
    # from up to bottom
    # from left to right
    # cannot be simply geenrated mgrid
    #hs, vs = np.mgrid[h_0+lon_cstep/2.: h_e: lon_cstep, v_e-lat_cstep/2.: v_0-lat_cstep: lat_cstep]

    #h_array = hs.ravel()
    #v_array = vs.ravel()

    # calculate the lat and lon for one MODIS tile
    #hs = np.arange(h_0, h_e-lon_cstep, lon_cstep)
    #vs = np.arange(v_e, v_0+lat_cstep, lat_cstep)[::-1]

    hs = np.arange(h_0,h_e,(h_e-h_0)/2400.)# The last coordinates should not included
    vs = np.arange(v_0,v_e,(v_e-v_0)/2400.)# since it belongs to the next one
    h_array = np.tile(hs, 2400).reshape(2400,2400).ravel()
    v_array = (np.tile(vs, 2400).reshape(2400,2400).T).ravel()
    
    h_array = np.tile(hs, 2400).reshape(2400,2400).ravel()
    v_array = (np.tile(vs, 2400).reshape(2400,2400).T).ravel()


    #transformation to the wgs
    wgs = tx.TransformPoints(zip(h_array,v_array))
    
    return wgs

In [15]:
get_wgs(27,5)

[(117.4866560283317, 39.99999999940561, 0.0),
 (117.49209522537167, 39.99999999940561, 0.0),
 (117.49753442241166, 39.99999999940561, 0.0),
 (117.50297361945164, 39.99999999940561, 0.0),
 (117.50841281649161, 39.99999999940561, 0.0),
 (117.51385201353159, 39.99999999940561, 0.0),
 (117.51929121057155, 39.99999999940561, 0.0),
 (117.52473040761153, 39.99999999940561, 0.0),
 (117.5301696046515, 39.99999999940561, 0.0),
 (117.53560880169151, 39.99999999940561, 0.0),
 (117.54104799873147, 39.99999999940561, 0.0),
 (117.54648719577148, 39.99999999940561, 0.0),
 (117.55192639281145, 39.99999999940561, 0.0),
 (117.55736558985143, 39.99999999940561, 0.0),
 (117.56280478689139, 39.99999999940561, 0.0),
 (117.56824398393137, 39.99999999940561, 0.0),
 (117.57368318097134, 39.99999999940561, 0.0),
 (117.57912237801133, 39.99999999940561, 0.0),
 (117.58456157505131, 39.99999999940561, 0.0),
 (117.59000077209129, 39.99999999940561, 0.0),
 (117.59543996913126, 39.99999999940561, 0.0),
 (117.600879166

In [12]:
np.arange(4447802.079,3335851.559,(3335851.559-4447802.079)/2400.)[-1]- 3335851.559

463.31271680491045

In [ ]:
Upper Left  (10007554.677, 4447802.079) (117d29'11.96"E, 40d 0' 0.00"N)
Lower Left  (10007554.677, 3335851.559) (103d55'22.97"E, 30d 0' 0.00"N)
Upper Right (11119505.197, 4447802.079) (130d32'26.62"E, 40d 0' 0.00"N)
Lower Right (11119505.197, 3335851.559) (115d28'12.19"E, 30d 0' 0.00"N)